In [4]:
import cv2
import PIL
from skimage.transform import resize
from tensorflow.keras.layers import Input, Conv2D,BatchNormalization,LeakyReLU, ZeroPadding2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate,Add
from tensorflow.keras.preprocessing.image import load_img, img_to_array,array_to_img
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
import struct
import scipy.misc
import os

In [5]:
class WeightReader:
    def __init__(self, weight_file):
        '''
        가중치 파일 원시 바이너리 형식으로 열고,
        파일의 헤더 정보를 읽어 들임
        '''
        # major, minor, revision 파일의 버전
        with open(weight_file, 'rb') as w_f:
            major, = struct.unpack('i', w_f.read(4))
            minor, = struct.unpack('i', w_f.read(4))
            revision ,= struct.unpack('i', w_f.read(4))

            # 파일 버전이 2 이상인 경우 두 개의 정수를 더 읽어들임

            if (major * 10 + minor) >= 2 and major <1000 and minor <1000:
                w_f.read(8)
            else:
                w_f.read(4)

            # major나 minor가 1000을 초과하면, transpose 를 True 로 설정
            transpose = (major > 1000) or (minor > 1000)

            binary = w_f.read()

        # 현재 파일에서 읽고 있는 위치
        self.offset = 0
        self.all_weights = np.frombuffer(binary , dtype='float32')


    def read_bytes(self, size):
        self.offset = self.offset + size
        return self.all_weights[self.offset - size : self.offset]

    def load_weight(self, model):
        for i in range(106):
            try:
                # 'conv_{i}' 레이어 찾기
                conv_layer = model.get_layer(f'conv_{str(i)}')
                # batchnorm 정규화 가중치 로드 (잠시후 에)
                if i not in [1,11,111]:
                    norm_layer = model.get_layer(f'bnorm_{str(i)}')
                    size = np.prod(norm_layer.get_weights()[0].shape)

                    beta = self.read_bytes(size) # 편향
                    gamma = self.read_bytes(size) # 스케일
                    mean = self.read_bytes(size) # 평균
                    var = self.read_bytes(size) # 분산
                    weights = norm_layer.set_weights([gamma, beta, mean, var])

                if len(conv_layer.get_weights())>1:
                    bias = self.read_bytes(np.prod(conv_layer.get_weights()[1].shape)) # 편향 값 가중
                    kernel = self.read_bytes(np.prod(conv_layer.get_weights()[0].shape)) # 커널 값 가중치
                    kernel = kernel.reshape(list(reversed(conv_layer.get_weights()[0].shape)))#
                    kernel = kernel.transpose([2,3,1,0]) # 가중치 transpose, custom transpose를 취함
                    conv_layer.set_weights([kernel, bias])
                else:
                    kernel = self.read_bytes(np.prod(conv_layer.get_weights()[0].shape)) # 커널 값 가중치
                    kernel = kernel.reshape(list(reversed(conv_layer.get_weights()[0].shape)))#
                    kernel = kernel.transpose([2,3,1,0]) # 가중치 transpose, custom transpose를 취함
                    conv_layer.set_weights([kernel])
            except ValueError:
                print(f'{str(i)} 해당 위치에 레이어가 없습니다')
    def reset(self):
        self.offset = 0

In [6]:
def _conv_block(inp, convs, skip=True):
    x = inp
    count = 0
    for conv in convs:
        # 블록 내 마지막 -1 번째 레이어가 skip 연결이 되어 있다면
        # 그 위치를 저장
        if count == (len(convs) -2) and skip:
            skip_connection = x
        count += 1
        # 다크넷은 패딩을 위쪽과 왼쪽에 선호 하기 때문에
        # stride가 1보다 큰 경우 위, 왼쪽에만 패딩을 씌움
        if conv['stride'] > 1: x = ZeroPadding2D(((1,0), (1,0)))(x)
        x = Conv2D(conv['filter'], conv['kernel'],
                   strides=conv['stride'],
                   padding = 'valid' if conv['stride'] > 1 else 'same',
                   name = 'conv_'+str(conv['layer_idx']),
                   use_bias=False if conv['bnorm'] else True)(x)

        #배치 정규화를 수행하는 레이어
        if conv['bnorm'] :
            x = BatchNormalization(epsilon=0.001, name='bnorm_'+str(conv['layer_idx']))(x)
        # 활성화 함수로 leaky_relu를 사용 하는 경우
        if conv['leaky'] :
            x = LeakyReLU(alpha=0.1 , name='leaky_'+str(conv['layer_idx']))(x)
    # skip 연결이 활성화가 되어 있다면 해당 위치의 텐서와 현재 텐서를 더한다.
    return Add()([skip_connection,x]) if skip else x



In [7]:
def make_yolov3_model():
    input_image = Input(shape=(None, None, 3))

    #0 ~ 4
    x = _conv_block(input_image,[{'filter':32, 'kernel' : 3, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':0},
                                 {'filter':64, 'kernel' : 3, 'stride' : 2, 'bnorm':True, 'leaky':True, 'layer_idx':1},
                                 {'filter':32, 'kernel' : 1, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':2},
                                 {'filter':64, 'kernel' : 3, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':3}])
    # 5 ~ 8
    x = _conv_block(x ,[{'filter':128, 'kernel' : 3, 'stride' : 2, 'bnorm':True, 'leaky':True, 'layer_idx':5},
                        {'filter':64, 'kernel' : 1, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':6},
                        {'filter':128, 'kernel' : 3, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':7}])
    # 9 ~ 11
    x = _conv_block(x ,[{'filter':64, 'kernel' : 1, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':9},
                        {'filter':128, 'kernel' : 3, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':10}])
    # 12 ~ 15
    x = _conv_block(x ,[{'filter':256, 'kernel' : 3, 'stride' : 2, 'bnorm':True, 'leaky':True, 'layer_idx':12},
                        {'filter':128, 'kernel' : 1, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':13},
                        {'filter':256, 'kernel' : 3, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':14}])
    # 16 ~ 36
    for i in range(7):
        #16 17 19 20 22 23 24 25
        x = _conv_block(x,[{'filter':128, 'kernel' : 1, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':16 + i * 3},
                        {'filter':256, 'kernel' : 3, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':17 + i * 3}])
    skip_36 = x

    # 37~40
    x = _conv_block(x ,[{'filter':512, 'kernel' : 3, 'stride' : 2, 'bnorm':True, 'leaky':True, 'layer_idx':37},
                        {'filter':256, 'kernel' : 1, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':38},
                        {'filter':512, 'kernel' : 3, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':39}])

    # 41 ~ 61
    for i in range(7):
        x = _conv_block(x,[{'filter':256, 'kernel' : 1, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':41 + i * 3},
                        {'filter':512, 'kernel' : 3, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':42 + i * 3}])
    skip_61 = x
    # 62~ 65
    x = _conv_block(x ,[{'filter':1024, 'kernel' : 3, 'stride' : 2, 'bnorm':True, 'leaky':True, 'layer_idx':62},
                        {'filter':512, 'kernel' : 1, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':63},
                        {'filter':1024, 'kernel' : 3, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':64}])
    # 66 ~74
    for i in range(3):
        x = _conv_block(x,[{'filter':512, 'kernel' : 1, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':66 + i * 3},
                        {'filter':1024, 'kernel' : 3, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':67 + i * 3}])
    # 75 ~ 79
    x = _conv_block(x ,[{'filter':512, 'kernel' : 1, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':75},
                        {'filter':1024, 'kernel' : 3, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':76},
                        {'filter':512, 'kernel' : 1, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':77},
                        {'filter':1024, 'kernel' : 3, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':78},
                        {'filter':512, 'kernel' : 1, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':79}], skip=False)

    yolo_82 = _conv_block(x ,[{'filter':1024, 'kernel' : 3, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':80},
                        {'filter':255, 'kernel' : 1, 'stride' : 1, 'bnorm':False, 'leaky':False, 'layer_idx':81}], skip=False)
    #
    x  =  _conv_block(x ,[{'filter':256, 'kernel' : 1, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':84}], skip=False)
    x = UpSampling2D(2)(x)
    x = concatenate([x, skip_61])

    #87 ~ 91
    x = _conv_block(x ,[{'filter':256, 'kernel' : 1, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':87},
                        {'filter':512, 'kernel' : 3, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':88},
                        {'filter':256, 'kernel' : 1, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':89},
                        {'filter':512, 'kernel' : 3, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':90},
                        {'filter':256, 'kernel' : 1, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':91}],skip=False)

    # 92 ~ 94
    yolo_94 = _conv_block(x ,[{'filter':512, 'kernel' : 3, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':92},
                        {'filter':255, 'kernel' : 1, 'stride' : 1, 'bnorm':False, 'leaky':False, 'layer_idx':93}],skip=False)

    x =  _conv_block(x ,[{'filter':128, 'kernel' : 1, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':96}], skip=False)
    x = UpSampling2D(2)(x)
    x = concatenate([x, skip_36])

    # 99 ~ 106
    yolo_106 = _conv_block(x ,[{'filter':128, 'kernel' : 1, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':99},
                        {'filter':256, 'kernel' : 3, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':100},
                        {'filter':128, 'kernel' : 1, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':101},
                        {'filter':256, 'kernel' : 3, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':102},
                        {'filter':128, 'kernel' : 1, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':103},
                        {'filter':256, 'kernel' : 3, 'stride' : 1, 'bnorm':True, 'leaky':True, 'layer_idx':104},
                        {'filter':255, 'kernel' : 1, 'stride' : 1, 'bnorm':False, 'leaky':False, 'layer_idx':105}],skip=False)

    model = Model(input_image, [yolo_82, yolo_94, yolo_106])
    return model

In [8]:
# yolo v3 모델 정의하기
yolov3 = make_yolov3_model()
yolov3.summary()

In [ ]:
!unzip /content/drive/MyDrive/yolo.zip

In [ ]:
weight_reader = WeightReader('/content/yolov3.weights')

In [ ]:
weight_reader.load_weight(yolov3)
yolov3.save('model.tf')

In [10]:
# 시그모이드 함수
def _sigmoid(x):
    return 1./(1+np.exp(-x))

In [ ]:
# 바운딩박스 세팅 클래스
class BoundBox:
    def __init__(self, xmin, ymin, xmax, ymax, objness = None, classes=None):
        self.xmin = xmin
        self.ymin = ymin
        self.xmax = xmax
        self.ymax = ymax
        # objness = 오브젝트 일 확률
        self.objness = objness
        # classes = 오브젝트가 어떤 클래스일지 확률
        self.classes = classes
        # 초기 레이블과 점수는 -1
        self.label = -1
        self.score = -1

    def get_label(self):
        # 레이블이 아직 설정되지 않았다면, 가장 높은 점수의 클래스를 레이블로 설정
        if self.label == -1:
            self.label = np.argmax(self.classes)
        return self.label

    def get_score(self):
        # 점수가 아직 설정되지 않았다면, 레이블 클래스 확률을 점수로 설정
        if self.score == -1:
            self.score = self.classes[self.get_label()]
        return self.score

In [ ]:
'''
네트 워크 출력을 디코딩 하는 함수
각 출력박스를 바운드 박스 객체로 변환
네트워크 출력, 앵커, 객체 입력값, 네트워크 높이 및 네트워크의 너비를 매개변수로 받는다.
출력은 바운딩 박스의 리스트,  각 x,y좌표, 객체일 확률, 클래스 확률값들이 나옴
'''
def decode_netout(netout, anchors, obj_thresh, net_h, net_w):
    grid_h,grid_w = netout.shape[:2]
    nb_box = 3
    netout = netout.reshape((grid_h, grid_w, nb_box, -1))
    boxes = []
    netout[... , :2] = _sigmoid(netout[... :2])
    # ...은 모든 그리드 셀을 의미, 0~1 depth 는 xy 0~1 normalize
    netout[... , 2:4] = _sigmoid(netout[... ,2:4])
    # ...은 모든 그리드 셀을 의미, 0~1 depth 는 wh 0~1 normalize
    netout[... , 5:] = netout[... , 4][... , np.newaxis] * netout[... , 5:]
    # objtness score * class probabilty
    netout[... , 5:] *= netout[... , 5:] > obj_thresh
    # 위 곱한값이 objthsh를 못넘으면 0으로 만듬

    for i in range(grid_h * grid_w):
        row = i / grid_w
        col = i % grid_w
        for b in range(nb_box):
            # 4번째 요소는 objectness 점수
            objectness = netout[int(row)][int[col]][b][4]
            # objectness가 임계값 보다 낮으면 해당 박스는 애초에 무시한다.
            if(objectness.all() < obj_thresh):continue
            # 처음 4개 요소는 x,y,w,h
            x,y,w,h = netout[int(row)][int[col]][b][:4]
            x = (col + x) / grid_w # yolo는 바운딩 박스의 좌상단 좌표가 아닌, 중앙값 기준
            y = (row + y) / grid_h # 중심위치, 단위: 이미지 높이
            w = anchors[2 * b + 0] * np.exp(w) / net_w
            h = anchors[2 * b + 1] * np.exp(h) / net_h

            # 마지막 요소들은 클래스 확률 입니다.
            classes = netout[int(row)][int(col)][b][5:]
            box = BoundBox(x,y,w,h,objectness, classes)
            boxes.append(box)
    return boxes
